In [1]:
from geoband.API import *
import pandas as pd

import folium
import geopandas as gpd

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data1 = pd.read_csv('new_data/data1_osan.csv')
data2_link_notzero = gpd.read_file('new_data/data2_link_notzero.geojson')
data9 = gpd.read_file('new_data/data9_buffer.geojson')

In [3]:
data2_link_notzero.tail()

,gid,accident_cnt,link_cnt,주정차단속수,geometry
291,다사633033,1,4.0,2,"MULTIPOLYGON (((127.08684 37.12767, 127.08683 ..."
292,다사633049,1,2.0,0,"MULTIPOLYGON (((127.08676 37.14209, 127.08676 ..."
293,다사634036,1,1.0,0,"MULTIPOLYGON (((127.08795 37.13038, 127.08794 ..."
294,다사635067,1,1.0,0,"MULTIPOLYGON (((127.08892 37.15832, 127.08892 ..."
295,다사636036,1,2.0,0,"MULTIPOLYGON (((127.09020 37.13038, 127.09020 ..."


### 1. 오산시_주정차단속(2018~2020) ###

In [12]:
# 단속위치_경도 & 단속위치_위도에 null값 존재
data1 = data1.dropna().reset_index(drop = True)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60846 entries, 0 to 60845
Data columns (total 7 columns):
단속일자       60846 non-null int64
단속위치       60846 non-null object
행정구역       60846 non-null object
스쿨존여부      60846 non-null object
단속위치_경도    60846 non-null float64
단속위치_위도    60846 non-null float64
단속종류       60846 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 3.2+ MB


In [13]:
data1.tail()

,단속일자,단속위치,행정구역,스쿨존여부,단속위치_경도,단속위치_위도,단속종류
60841,20201111,GS25궐동대호점 맞은편,경기도 오산시 청학동,N,127.053064,37.158140,고정형CCTV단속
60842,20201111,물향기지하도,경기도 오산시 원동,N,127.066157,37.171143,고정형CCTV단속
60843,20201111,운암2단지 근린공원삼거,경기도 오산시 청호동,N,127.077283,37.158126,고정형CCTV단속
60844,20201111,궐동 양평해장국 사거리,경기도 오산시 궐동,N,127.056665,37.155827,고정형CCTV단속
60845,20201111,어울림(북개천) 사거리,경기도 오산시 부산동,N,127.068644,37.143810,고정형CCTV단속


In [10]:
# geometry column (Point) 생성 함수 => make_geom(데이터, 경도, 위도)
def make_geom(df, lon, lat):
    geom = gpd.points_from_xy(df[lon], df[lat])
    df = gpd.GeoDataFrame(df, geometry = geom)
    df.crs = {'init' : 'epsg:4326'}
    return df

In [15]:
data1 = make_geom(data1, '단속위치_경도', '단속위치_위도')
data1.tail()

,단속일자,단속위치,행정구역,스쿨존여부,단속위치_경도,단속위치_위도,단속종류,geometry
60841,20201111,GS25궐동대호점 맞은편,경기도 오산시 청학동,N,127.053064,37.158140,고정형CCTV단속,POINT (127.0530642 37.15814039)
60842,20201111,물향기지하도,경기도 오산시 원동,N,127.066157,37.171143,고정형CCTV단속,POINT (127.066157 37.17114288)
60843,20201111,운암2단지 근린공원삼거,경기도 오산시 청호동,N,127.077283,37.158126,고정형CCTV단속,POINT (127.0772832 37.15812564)
60844,20201111,궐동 양평해장국 사거리,경기도 오산시 궐동,N,127.056665,37.155827,고정형CCTV단속,POINT (127.056665 37.155827)
60845,20201111,어울림(북개천) 사거리,경기도 오산시 부산동,N,127.068644,37.143810,고정형CCTV단속,POINT (127.0686441 37.14380963)


In [9]:
# df2의 격자내에 존재하는 df1의 POINT 개수를 저장하는 함수
# col : 저장할 column 이름
def contain_count(df1, df2, col):
    df2[col] = 0
    for idx1 in df1.index:
        for idx2 in df2.index:
            if df1.geometry[idx1].within(df2.geometry[idx2]):
                df2[col][idx2] += 1

In [17]:
contain_count(data1, data2_link_notzero, '주정차단속수')

In [18]:
data2_link_notzero

,gid,accident_cnt,link_cnt,geometry,주정차단속수
0,다사561083,1,2.0,(POLYGON ((127.0054916936862 37.17242713739633...,0
1,다사564096,1,2.0,"(POLYGON ((127.0087950432604 37.184159165098, ...",0
2,다사566102,1,1.0,(POLYGON ((127.0110133334056 37.18957677430988...,0
3,다사583093,1,1.0,(POLYGON ((127.0302168407684 37.18154181816758...,0
4,다사584072,1,7.0,(POLYGON ((127.0314602779681 37.16261716235583...,0
5,다사586106,1,2.0,(POLYGON ((127.0335244040752 37.19327316786908...,26
6,다사587072,1,4.0,"(POLYGON ((127.0348390682982 37.1626304727788,...",0
7,다사587098,1,1.0,(POLYGON ((127.0346953708479 37.18606652474183...,0
8,다사588096,1,4.0,(POLYGON ((127.0358330154574 37.18426817367395...,0
9,다사589069,1,2.0,(POLYGON ((127.0371080859615 37.15993512616837...,53


In [30]:
data2_link_notzero.to_file('new_data/data2_link_notzero.geojson', driver='GeoJSON')

### 9. 오산시_어린이보호구역 ###

In [8]:
data9.tail()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도,geometry
86,어린이집,시립세마어린이집,Y,1,12,127.044916,37.184608,"POLYGON ((127.04548 37.18461, 127.04548 37.184..."
87,어린이집,미래숲어린이집,Y,1,6,127.044384,37.145326,"POLYGON ((127.04495 37.14533, 127.04494 37.145..."
88,어린이집,시립은여울어린이집,N,0,20,127.066860,37.170952,"POLYGON ((127.06742 37.17095, 127.06742 37.170..."
89,어린이집,설리반어린이집,N,0,5,127.061041,37.164127,"POLYGON ((127.06160 37.16413, 127.06160 37.164..."
90,어린이집,세교복지타운어린이집,Y,1,15,127.044835,37.177752,"POLYGON ((127.04540 37.17775, 127.04540 37.177..."


In [3]:
def intersects_count(df1, df2, col):
    df2[col] = 0
    for idx1 in df1.index:
        for idx2 in df2.index:
            if df1.geometry[idx1].intersects(df2.geometry[idx2]):
                df2[col][idx2] += 1

In [8]:
intersects_count(data9, data2_link_notzero, '어린이보호구역수')

In [5]:
data2_link_notzero['어린이보호구역수'].value_counts()

0    255
1     30
2     11
Name: 어린이보호구역수, dtype: int64

In [9]:
data2_link_notzero['어린이보호구역'] = 'N'
for idx in data2_link_notzero.index:
    if data2_link_notzero['어린이보호구역수'][idx] != 0:
        data2_link_notzero['어린이보호구역'][idx] = 'Y'

In [10]:
data2_link_notzero.drop(columns = '어린이보호구역수', inplace = True)
print(data2_link_notzero['어린이보호구역'].value_counts())
data2_link_notzero.tail()

N    255
Y     41
Name: 어린이보호구역, dtype: int64


,gid,accident_cnt,link_cnt,주정차단속수,geometry,어린이보호구역
291,다사633033,1,4.0,2,"MULTIPOLYGON (((127.08684 37.12767, 127.08683 ...",N
292,다사633049,1,2.0,0,"MULTIPOLYGON (((127.08676 37.14209, 127.08676 ...",N
293,다사634036,1,1.0,0,"MULTIPOLYGON (((127.08795 37.13038, 127.08794 ...",N
294,다사635067,1,1.0,0,"MULTIPOLYGON (((127.08892 37.15832, 127.08892 ...",N
295,다사636036,1,2.0,0,"MULTIPOLYGON (((127.09020 37.13038, 127.09020 ...",N


In [11]:
gdf = gpd.overlay(data9, data2_link_notzero, how='intersection')

In [12]:
m = folium.Map(location = [37.149799, 127.077116], zoom_start = 12)

folium.Choropleth(geo_data = gdf, 
                  data = gdf,
                  columns = ('gid', 'accident_cnt'), 
                  key_on='feature.properties.gid',
                  fill_color = 'PuRd', 
                  legend_name='교통사고수').add_to(m)
m

In [ ]:
data2_link_notzero.to_file('new_data/data2_link_notzero.geojson', driver='GeoJSON')